In [ ]:
import pandas as pd
import numpy as np
import addfips
import plotly.express as px
from urllib.request import urlopen
import json
import math


In [ ]:
charger_amount_df = pd.read_csv('EV Chargers_Last updated 01-31-2022.csv')

In [ ]:
charger_amount_df = charger_amount_df.dropna()

In [ ]:
af = addfips.AddFIPS()

def create_fips_col(county):
    fips = af.get_county_fips(county, state = 'California')
    return fips

# add 'fips' column to each df
charger_amount_df['fips'] = charger_amount_df['County'].apply(create_fips_col)

In [ ]:
charger_amount_df['log_Total'] = charger_amount_df['Total'].apply(math.log)

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

target_states = ['06']
counties['features'] = [f for f in counties['features'] if f['properties']['STATE'] in target_states]    

In [ ]:
charger_fig = px.choropleth(
    charger_amount_df,
    geojson = counties,
    locations = 'fips',
    color = 'log_Total',
    scope = 'usa',
    color_continuous_scale= 'agsunset_r',
    hover_name = charger_amount_df['County'],
    hover_data= {'Total': True,
                 'fips': False,
                 'log_Total': False
                 },
    basemap_visible=False
           
)

charger_fig.update_geos(fitbounds = 'locations')
charger_fig.update_layout(height=500,margin={"r":0,"t":0,"l":0,"b":0})
charger_fig.layout.coloraxis.colorbar = {'title': 'Number of EV Chargers',
                                 'x': 0.9,
                                 'tickvals': [0,1,2,3,4,5,6,7,8,9,10, 11],
                                 'ticktext': [0, 2, 5] + [str(int(round(math.exp(val), -1))) for val in range(2,11)]}
charger_fig.show()